In [10]:
from toolkit import *

In [12]:
daily_data_run()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\kacpe\anaconda3\Lib\site-packages\seaborn\matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
C:\Users\kacpe\anaconda3\Lib\site-packages\ydata_profiling\model\missing.py:78: UserWarning: There was an attempt to generate the Heatmap missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Heatmap": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: '--'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
sleep_length = 2
search_for_inactive_destination_path = 'data_raw/search_for_inactive'
get_links_destination_path= 'data_raw/available_ads'
scrape_otodom_destination_path = 'data_raw/otodom_scraped_data'
report_destination_path = 'data_raw/data_reports'
main_path = 'data_processed/main.csv'

main = pd.read_csv(main_path)
new_announcements = pd.read_csv('data_raw/available_ads/available_ads_2025_02_06.csv')
driver = initialize_otodom_scraper(2)
links = list(set(new_announcements.link)-set(main.link))

new_records = scrape_otodom_announcements(
    driver = driver,
    announcements_links = links,
    sleep_length = sleep_length,
    csv_destination_path = scrape_otodom_destination_path)

driver.close()

new_records = process_data(new_records)
new_records = new_records[~new_records.link.isin(main.link)]

# Concat main and newly scraped announcements
main = pd.concat([main, new_records], ignore_index=True)

# Save main
main.to_csv(main_path,
        encoding='utf-8',
        index=False)

# Create profile report for newly scraped announcements
current_date = datetime.now().strftime('%Y_%m_%d')
profile = ProfileReport(new_records, title='Data Profiling Report')
profile.to_file(f'{report_destination_path}/scraped_announcements_{current_date}.html')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00007FF6586502F5+28725]
	(No symbol) [0x00007FF6585B2AE0]
	(No symbol) [0x00007FF65844510A]
	(No symbol) [0x00007FF65841EEA5]
	(No symbol) [0x00007FF6584C6F87]
	(No symbol) [0x00007FF6584DFA52]
	(No symbol) [0x00007FF6584BFD53]
	(No symbol) [0x00007FF65848A0E3]
	(No symbol) [0x00007FF65848B471]
	GetHandleVerifier [0x00007FF65897F30D+3366989]
	GetHandleVerifier [0x00007FF6589912F0+3440688]
	GetHandleVerifier [0x00007FF6589878FD+3401277]
	GetHandleVerifier [0x00007FF65871AAAB+858091]
	(No symbol) [0x00007FF6585BE74F]
	(No symbol) [0x00007FF6585BA304]
	(No symbol) [0x00007FF6585BA49D]
	(No symbol) [0x00007FF6585A8B69]
	BaseThreadInitThunk [0x00007FF9D261E8D7+23]
	RtlUserThreadStart [0x00007FF9D337FBCC+44]


In [ ]:
len(links)

In [ ]:
import pandas as pd
from scraper import *
from datakit import *
from datetime import datetime
from ydata_profiling import ProfileReport

In [ ]:
def daily_data_run():

# Read main
path = f'data_processed/main.csv'
main = pd.read_csv(path)

### TODO Add rent price updates
# Search for inactive
inactive = search_for_inactive(
    list(main[main.expired.eq(0)].link),
    csv_destination_path = search_for_inactive_destination_path.
    sleep_length=1)
inactive = inactive[inactive.expired.eq(1)]

# Update main with inactive
main.update(main[['link']].merge(inactive, on='link', how='left'), overwrite=True)

# Search for new announcement links
new_announcements_url = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?limit=36&daysSinceCreated=7&by=DEFAULT&direction=DESC&viewType=listing&page=2'
new_announcements = get_links_titles(new_announcements_url, csv_destination_path = get_links_destination_path)

# Scrape new announcements
driver = initialize_otodom_scraper(2)
links = list(set(df.new_announcements)-set(main.link))

new_records = scrape_otodom_announcements(
    driver = driver,
    announcements_links = links,
    sleep_length = 2)

driver.close()

# Process scrabed announcements
new_records = process_data(new_records)
new_records = new_records[~new_records.link.isin(main.link)]

# Concat main and newly scraped announcements
main = pd.concat([main, df], ignore_index=True)

# Save main
main.to_csv(path,
          encoding='utf-8',
          index=False)

# Create profile report for newly scraped announcements
current_date = datetime.now().strftime('%Y_%m_%d')
profile = ProfileReport(new_records, title='Data Profiling Report')
profile.to_file(f'data_raw/data_reports/scraped_announcements_{current_date}.html')

In [ ]:
new_records = pd.read_csv('data_raw/search_for_inactive/search_for_inactive_2024_12_29.csv')
current_date = datetime.now().strftime('%Y_%m_%d')
profile = ProfileReport(new_records, title='Data Profiling Report')
profile.to_file(f'data_raw/data_reports/scraped_announcements_{current_date}.html')